In [1]:
# import das libs necessárias
import pandas as pd # trabalhar com dataframes
import numpy as np # realizacao de algumas operacoes com matrizes

#imagens
import cv2 # transformacoes faceis em imagens
from PIL import Image # trabalhar com imagens

# ferramentas
import glob # exploracao de diretorios
from pylab import *
import tqdm
import importlib
import time

# plot 
import matplotlib.pyplot as plt # plotagem
%matplotlib inline

# Machine Learning
from sklearn.neural_network import MLPClassifier 
from sklearn.model_selection import GridSearchCV # Grid Search para rodar todos os parametros

#importando ferramentas já implementadas anteriormente
import sampling #retorna uma amostra para treino
importlib.reload(sampling)
import PCA_and_Inverse
importlib.reload(PCA_and_Inverse)
import feature_selection
importlib.reload(feature_selection)

<module 'feature_selection' from '/home/fernando.favoretti/src/personal/pattern_recognition/MLP/feature_selection.py'>

## Cria uma amostra das imagens do tamanho definido e carrega em dataframes

In [17]:
#carregando amostra dos dados
X, Y = sampling.get_sample(1000)

100%|██████████| 1000/1000 [00:01<00:00, 543.87it/s]


## Normaliza os dados

In [18]:
X = sampling.get_min_max_normalization(data=X)

## Extração de características

- Calcula as Componentes principais para os dados carregados

In [4]:
pca = PCA_and_Inverse.PCA_and_Inverse()
pca_components = pca.get_PCA(X=X, n_components=0.9)

## Seleção de características

- Calcula as melhores features a serem utilizadas pelos métodos de:
    - Informação Mútua
    - Algoritmo Genético

#### Informação Mútua

In [5]:
mic_threshold = 0.05
mic_features_df = feature_selection.get_MIC_features(X=X, Y=Y)
mic_features = list(mic_features_df[mic_features_df[0] > mic_threshold].index)

/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#### Algoritmo Genético

In [6]:
ga_features = feature_selection.get_GA_features(X=X, Y=Y)
ga_features

Training Features Shape: (750, 160000)
Training Labels Shape: (750, 1)
Testing Features Shape: (250, 160000)
Testing Labels Shape: (250, 1)
Baseline AUC 0.6146260245901639
Optimizing with 1000 generations
Generation 0
Best result : 0.595671106557377
Generation 1
Best result : 0.6003457991803279
Generation 2
Best result : 0.6083504098360656
Generation 3
Best result : 0.6083504098360656
Generation 4
Best result : 0.6173155737704918
Generation 5
Best result : 0.6173155737704918
Generation 6
Best result : 0.6173155737704918
Generation 7
Best result : 0.6173155737704918
Generation 8
Best result : 0.6173155737704918
Generation 9
Best result : 0.6173155737704918
Generation 10
Best result : 0.6173155737704918
Generation 11
Best result : 0.6173155737704918
Generation 12
Best result : 0.6173155737704918
Generation 13
Best result : 0.6173155737704918
Generation 14
Best result : 0.6173155737704918
Generation 15
Best result : 0.6173155737704918
Generation 16
Best result : 0.6304431352459017
Generat

[89690,
 56515,
 74897,
 61876,
 150001,
 136787,
 11141,
 29858,
 40916,
 155454,
 111048,
 107654,
 34890,
 33819,
 31899,
 141599,
 121830,
 51623,
 45065,
 39579,
 3638,
 7857,
 19668,
 65443,
 57583,
 66752,
 149308,
 112355,
 97378,
 129756,
 24064,
 57387,
 47643,
 139877,
 146829,
 122017,
 40625,
 30022,
 131537,
 128925,
 24646,
 137062,
 155134,
 132947,
 157199,
 97102,
 6342,
 86065,
 60728,
 18348,
 8279,
 57409,
 87484,
 10129,
 62481,
 130633,
 126666,
 111908,
 116517,
 82151,
 47092,
 35381,
 65027,
 39613,
 94264,
 119341,
 79991,
 37794,
 152997,
 46226,
 94269,
 78682,
 95116,
 42557,
 8443,
 101592,
 100340,
 112025,
 4722,
 41445,
 157553,
 82027,
 61556,
 36197,
 24498,
 19220,
 32261,
 101641,
 134288,
 125738,
 86788,
 8750,
 39724,
 114007,
 26240,
 138048,
 143943,
 44848,
 43342,
 89096,
 100076,
 53091,
 97302,
 91915,
 99675,
 50619,
 91181,
 89593,
 36085,
 5328,
 104191,
 17738,
 61625,
 104611,
 135879,
 152313,
 132725,
 61348,
 51628,
 22559,
 63227,

## Gera as combinações de parâmetros para rodar a rede neural

In [7]:
mlp_classifier = MLPClassifier(solver="sgd", learning_rate="constant")

parameters = [{"hidden_layer_sizes":[(10,), (25,), #variando o tamanho de uma única camada escondida
                                     (10, 15), # duas camadas ocultas com 10 e 15 neuronios cada
                                     (10, 10, 10)], # três camadas ocultas com 10 neuronios cada
               "learning_rate_init":[0.001, 0.01, 0.1, 0.5, 0.9],
               "activation":['logistic', 'relu']}]
scoring = ["accuracy", "precision", "recall", "roc_auc"]

result_column_list= ['param_activation',
                     'param_hidden_layer_sizes',
                     'param_learning_rate_init',
                     'mean_test_accuracy',
                     'mean_test_precision',
                     'mean_test_recall',
                     'mean_test_roc_auc']

#### Roda todas as combinações da rede neural para todas as features

In [20]:
grid_search_full = GridSearchCV(mlp_classifier, parameters, verbose=5, n_jobs=-1, cv=3, scoring=scoring, refit=False)

In [21]:
grid_search_full.fit(X, Y)

Fitting 3 folds for each of 40 candidates, totalling 120 fits
[CV] activation=logistic, hidden_layer_sizes=(10,), learning_rate_init=0.001 


/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] activation=logistic, hidden_layer_sizes=(10,), learning_rate_init=0.001 


Process ForkPoolWorker-75:
Process ForkPoolWorker-74:
Process ForkPoolWorker-73:
Process ForkPoolWorker-71:
Process ForkPoolWorker-67:
Traceback (most recent call last):
Process ForkPoolWorker-72:
Process ForkPoolWorker-69:
Process ForkPoolWorker-64:
Traceback (most recent call last):
Process ForkPoolWorker-70:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-65:
Process ForkPoolWorker-66:
Process ForkPoolWorker-63:
  File "/home/fernando.favoretti/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-68:
Traceback (most recent call last):
  File "/home/fernando.favoretti/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-76:
  File "/home/fernando.favoretti/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  

[CV]  activation=logistic, hidden_layer_sizes=(10,), learning_rate_init=0.001, accuracy=0.5119760479041916, precision=0.5061728395061729, recall=0.9820359281437125, roc_auc=0.5597188855821291, total=   3.1s


Process ForkPoolWorker-62:
Process ForkPoolWorker-61:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/fernando.favoretti/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/fernando.favoretti/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/fernando.favoretti/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/fernando.favoretti/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/fernando.favoretti/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/fernando.favoretti/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/externals/jobl

KeyboardInterrupt: 

In [19]:
full_result = pd.DataFrame(grid_search_full.cv_results_)[result_column_list]
full_result.to_csv("Sample_1000_Full.csv")

AttributeError: 'GridSearchCV' object has no attribute 'cv_results_'

#### Roda todas as combinações da rede neural para as features extraídas pelo PCA

In [22]:
grid_search_pca = GridSearchCV(mlp_classifier, parameters, verbose=5, n_jobs=-1, cv=3, scoring=scoring, refit=False)

In [23]:
grid_search_pca.fit(pca_components, Y)

Fitting 3 folds for each of 40 candidates, totalling 120 fits
[CV] activation=logistic, hidden_layer_sizes=(10,), learning_rate_init=0.001 
[CV] activation=logistic, hidden_layer_sizes=(10,), learning_rate_init=0.001 
[CV] activation=logistic, hidden_layer_sizes=(10,), learning_rate_init=0.001 
[CV] activation=logistic, hidden_layer_sizes=(10,), learning_rate_init=0.01 
[CV] activation=logistic, hidden_layer_sizes=(10,), learning_rate_init=0.01 


/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] activation=logistic, hidden_layer_sizes=(10,), learning_rate_init=0.01 


/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] activation=logistic, hidden_layer_sizes=(10,), learning_rate_init=0.1 


...

[CV]  activation=relu, hidden_layer_sizes=(10, 15), learning_rate_init=0.001, accuracy=0.5538922155688623, precision=0.5542168674698795, recall=0.5508982035928144, roc_auc=0.5699379683746281, total=   0.6s


[Parallel(n_jobs=-1)]: Done 114 out of 120 | elapsed:    3.9s remaining:    0.2s
/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV]  activation=relu, hidden_layer_sizes=(10, 10, 10), learning_rate_init=0.001, accuracy=0.4819277108433735, precision=0.48026315789473684, recall=0.4397590361445783, roc_auc=0.4854478153578168, total=   0.6s


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    4.2s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='sgd', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'hidden_layer_sizes': [(10,), (25,), (10, 15), (10, 10, 10)], 'learning_rate_init': [0.001, 0.01, 0.1, 0.5, 0.9], 'activation': ['logistic', 'relu']}],
       pre_dispatch='2*n_jobs', refit=False, return_train_score='warn',
       scoring=['accuracy', 'precision', 'recall', 'roc_auc'], verbose=5)

In [25]:
pd.DataFrame(grid_search_pca.cv_results_).max()

/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwa

mean_fit_time                 1.14041
std_fit_time                 0.456068
mean_score_time             0.0269552
std_score_time              0.0315324
param_activation                 relu
param_hidden_layer_sizes        (25,)
param_learning_rate_init          0.9
split0_test_accuracy         0.571856
split1_test_accuracy          0.58982
split2_test_accuracy         0.563253
mean_test_accuracy              0.555
std_test_accuracy            0.041043
rank_test_accuracy                 40
split0_train_accuracy               1
split1_train_accuracy               1
split2_train_accuracy               1
mean_train_accuracy                 1
std_train_accuracy           0.114733
split0_test_precision               1
split1_test_precision               1
split2_test_precision             0.6
mean_test_precision          0.716537
std_test_precision           0.282559
rank_test_precision                40
split0_train_precision              1
split1_train_precision              1
split2_train

In [26]:
pca_result = pd.DataFrame(grid_search_pca.cv_results_)[result_column_list]
pca_result.to_csv("Sample_1000_PCA.csv")

/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwa

#### Roda todas as combinações da rede neural para as features selecionadas pela Informação Mútua (MIC)

In [27]:
grid_search_mic = GridSearchCV(mlp_classifier, parameters, verbose=5, n_jobs=-1, cv=3, scoring=scoring, refit=False)

In [28]:
grid_search_mic.fit(X[mic_features], Y)

Fitting 3 folds for each of 40 candidates, totalling 120 fits
[CV] activation=logistic, hidden_layer_sizes=(10,), learning_rate_init=0.001 
[CV] activation=logistic, hidden_layer_sizes=(10,), learning_rate_init=0.001 


/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] activation=logistic, hidden_layer_sizes=(10,), learning_rate_init=0.001 
[CV] activation=logistic, hidden_layer_sizes=(10,), learning_rate_init=0.01 


/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] activation=logistic, hidden_layer_sizes=(10,), learning_rate_init=0.01 


...

[CV]  activation=relu, hidden_layer_sizes=(10, 15), learning_rate_init=0.01, accuracy=0.536144578313253, precision=0.5441176470588235, recall=0.4457831325301205, roc_auc=0.5408622441573523, total=   0.4s
[CV]  activation=relu, hidden_layer_sizes=(25,), learning_rate_init=0.01, accuracy=0.5449101796407185, precision=0.5348837209302325, recall=0.688622754491018, roc_auc=0.5537308616300334, total=   0.3s
[CV] activation=relu, hidden_layer_sizes=(25,), learning_rate_init=0.01 


/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV]  activation=relu, hidden_layer_sizes=(25,), learning_rate_init=0.01, accuracy=0.5598802395209581, precision=0.5515463917525774, recall=0.6407185628742516, roc_auc=0.5844956793000824, total=   0.3s


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    4.0s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='sgd', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'hidden_layer_sizes': [(10,), (25,), (10, 15), (10, 10, 10)], 'learning_rate_init': [0.001, 0.01, 0.1, 0.5, 0.9], 'activation': ['logistic', 'relu']}],
       pre_dispatch='2*n_jobs', refit=False, return_train_score='warn',
       scoring=['accuracy', 'precision', 'recall', 'roc_auc'], verbose=5)

In [29]:
mic_result = pd.DataFrame(grid_search_mic.cv_results_)[result_column_list]
mic_result.to_csv("Sample_1000_MIC.csv")

/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwa

#### Roda todas as combinações da rede neural para as features selecionadas pelo Algoritmo Genético (GA)

In [30]:
grid_search_ga = GridSearchCV(mlp_classifier, parameters, verbose=5, n_jobs=-1, cv=3, scoring=scoring, refit=False)

In [ ]:
grid_search_ga.fit(pca_components, Y)

Fitting 3 folds for each of 40 candidates, totalling 120 fits
[CV] activation=logistic, hidden_layer_sizes=(10,), learning_rate_init=0.001 
[CV] activation=logistic, hidden_layer_sizes=(10,), learning_rate_init=0.001 
[CV] activation=logistic, hidden_layer_sizes=(10,), learning_rate_init=0.001 
[CV] activation=logistic, hidden_layer_sizes=(10,), learning_rate_init=0.01 


/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] activation=logistic, hidden_layer_sizes=(10,), learning_rate_init=0.01 
[CV] activation=logistic, hidden_layer_sizes=(10,), learning_rate_init=0.01 


/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] activation=logistic, hidden_layer_sizes=(10,), learning_rate_init=0.1 
[CV] activation=logistic, hidden_layer_sizes=(10,), learning_rate_init=0.1 


...

/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV]  activation=relu, hidden_layer_sizes=(10,), learning_rate_init=0.01, accuracy=0.5179640718562875, precision=0.51875, recall=0.49700598802395207, roc_auc=0.5114919860877049, total=   0.6s
[CV]  activation=relu, hidden_layer_sizes=(10, 10, 10), learning_rate_init=0.1, accuracy=0.5149700598802395, precision=0.5107296137339056, recall=0.7125748502994012, roc_auc=0.5042489870558284, total=   0.1s
[CV]  activation=relu, hidden_layer_sizes=(10, 15), learning_rate_init=0.01, accuracy=0.5179640718562875, precision=0.5176470588235295, recall=0.5269461077844312, roc_auc=0.5286313600344221, total=   0.4s


/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV]  activation=relu, hidden_layer_sizes=(10, 10, 10), learning_rate_init=0.001, accuracy=0.5299401197604791, precision=0.5357142857142857, recall=0.4491017964071856, roc_auc=0.5178744307791603, total=   1.1s
[CV]  activation=relu, hidden_layer_sizes=(10, 10, 10), learning_rate_init=0.001, accuracy=0.5119760479041916, precision=0.5120481927710844, recall=0.5089820359281437, roc_auc=0.5193445444440461, total=   1.1s
[CV] activation=relu, hidden_layer_sizes=(10, 10, 10), learning_rate_init=0.001 


/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fernando.favoretti/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
ga_result = pd.DataFrame(grid_search_ga.cv_results_)[result_column_list]
ga_result.to_csv("Sample_400_GA.csv")